# Exploring Tweets data for monitoring Disaster using the exploratory data analysis (EDA) notebook

#### Introduction

This notebook is the first step in the DS4A Team 155 Project: Exploratory Data Analysis.
Social Networks have become a crucial communication channel in emergencies since users may report concerning situations in real-time. As a result, more agencies (i.e. disaster relief organizations and news agencies) are interested in monitoring Social Networks such as Twitter. However, tweets from people may or may not be actually realted to disasters. Therefore, machine learning models which predict real-time publication about emergencies and disasters are required.
From a set of different tweets, this project aims to predict which Tweets are about real disasters and which ones are not.


##### The proposed EDA approach to analysis the twitter messages is divided as follows: 
- Import the libraries
- Input Data
- Basic Informations
- Clean the data
- Feature Engineering
- Analizing Data
- Process Data
- Summary


### Import libraries

In [1]:
pip install -r requirements.txt --user

  Using cached folium-0.12.1.post1-py2.py3-none-any.whl (95 kB)
  Using cached jupyter_dash-0.4.2-py3-none-any.whl (23 kB)
  Using cached wordcloud-1.8.1.tar.gz (220 kB)


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Usuario.DESKTOP-GK240NU\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Usuario.DESKTOP-GK240NU\\AppData\\Local\\Temp\\pip-install-2_qrv0lu\\wordcloud_7dcd46b61c014b23831ef7ebb9720649\\setup.py'"'"'; __file__='"'"'C:\\Users\\Usuario.DESKTOP-GK240NU\\AppData\\Local\\Temp\\pip-install-2_qrv0lu\\wordcloud_7dcd46b61c014b23831ef7ebb9720649\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Usuario.DESKTOP-GK240NU\AppData\Local\Temp\pip-wheel-zoqa8fva'
       cwd: C:\Users\Usuario.DESKTOP-GK240NU\AppData\Local\Temp\pip-install-2_qrv0lu\wordcloud_7dcd46b61c014b23831ef7ebb9720649\
  Complete output (20 lines):
  running bdist_wh


  Running setup.py clean for wordcloud
Failed to build wordcloud
    Running setup.py install for wordcloud: started
    Running setup.py install for wordcloud: finished with status 'error'


  ERROR: Failed building wheel for wordcloud
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\Usuario.DESKTOP-GK240NU\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Usuario.DESKTOP-GK240NU\\AppData\\Local\\Temp\\pip-install-2_qrv0lu\\wordcloud_7dcd46b61c014b23831ef7ebb9720649\\setup.py'"'"'; __file__='"'"'C:\\Users\\Usuario.DESKTOP-GK240NU\\AppData\\Local\\Temp\\pip-install-2_qrv0lu\\wordcloud_7dcd46b61c014b23831ef7ebb9720649\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record 'C:\Users\Usuario.DESKTOP-GK240NU\AppData\Local\Temp\pip-record-0q79y2yn\install-record.txt' --single-version-externally-managed --user --prefix= --compile --install-headers 'C:\Users\Usuario.DESKTOP

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import nltk # imports the natural language toolkit
nltk.download('punkt')
nltk.download('stopwords')
import string
import plotly
import re

from nltk.stem import PorterStemmer 
from pylab import rcParams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from collections import Counter
from collections import defaultdict
from nltk.util import ngrams
# Importing the required parameter for plotting 
import matplotlib.pyplot as plt
from wordcloud import WordCloud

[nltk_data] Downloading package punkt to C:\Users\Usuario.DESKTOP-
[nltk_data]     GK240NU\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Usuario.DESKTOP-
[nltk_data]     GK240NU\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

### Input Data

The data for this project includes 10876 tweets, divided into two different datasets: a train dataset (7613 tweets) and a test dataset (3263 tweets)
The datasets have the following columns:


The train dataset has the following columns:
- id - a unique identifier for each tweet
- text - the text of the tweet
- location - the location the tweet was sent from (may be blank)
- keyword - a particular keyword from the tweet (may be blank)
- target - in train.csv only, this feature denotes whether a tweet is about a real disaster (1) or not (0)


#### Train dataset

In [ ]:
locations = pd.read_csv('Data/worldcities.csv')
locations = locations[["city", "country",'iso3']].astype(str)
locations['city']=locations['city'].str.lower()
locations['country']=locations['country'].str.lower()
locations


In [ ]:
df = pd.read_csv('Data/train.csv', dtype={'id': int, 'keyword': object, 'location': object, 'text': object, 'target': int})
df.head(5)

#### Test dataset

The test dataset is the verification values ​​of the prediction model on disasters and does not have the target column in order to validate if the result of the model on the analyzed text is correct (validation of the tweet)

In [ ]:
df_test = pd.read_csv('Data/test.csv', dtype={'id': int, 'keyword': object, 'location': object, 'text': object})
df_test.head(5)

### Basic Informations

 In this case we use the info() function to print a concise summary of a DataFrame.This method prints information about a DataFrame including the index dtype and column dtypes, non-null values and memory usage. For our case, we have 7613 rows and 5 columns. Location is the column with the most null values, followed by the Keywords column. Also, we can see that we have two data types: int32 and object.


In [ ]:
df.info( )

The describe() method returns a description of the data in the df. If the df contains numerical data, the description contains: count, mean, std, min, percentiles (25%,50%,75%) and max columns. 

In [ ]:
df.describe( )

In our DataFrame, we can find two numerical columns: id & target. For this case, we have complete information for the 7613 tweets (non null values). The mean of the target column is 0.42966, which means that most of the tweets do not correspond to a disaster (0: not disaster, 1: disaster), we can verify this information in the value counts function of the target column, that show us that we have 4342 with “0” target and 3271 with “1” target.

In [ ]:
df["target"].value_counts()

With the function “df.isnull().sum()” we are adding the number of null values ​​that we have in each column. As we mentioned before, the column with the most null values is location with 2533 null values, followed by the Keyword column with 61 null values.


In [ ]:
df.isnull().sum()

##### Observations

- There is a total of 7613 entries (train)
- There are numerous blanks in the dataset that need to be replaced with NA


### Clean Data

In the clean data part will be analyzed the characteristics of the information provided. This analysis is done based on the form of the data (uppercase, lowercase, etc.), the missing data, wrongly formatted data (for example, that in the text character cell it has a numeric data, they have special characters or links), the length of the tweet and how many words they have.


#### Removing rows, which own non-numerical values in the target column

In this part, we have a dataset that was classified by natural disaster based on a text provided by a tweet. To avoid having data that was not classified in the train part, we remove the data that has no classification with the .notnull() function.



In [ ]:
df[pd.to_numeric(df['target'], errors='coerce').notnull()]

In the target column, a value other than 1 and 0 may have been entered during the classification stage. In this step, the dataset is filtered only by the values ​​that have a 1 and a 0 in the target.

#### Removing rows, which have not 1 and 0 values in target column


In the target column, a value other than 1 and 0 may have been entered during the classification stage. In this step, the dataset is filtered only by the values ​​that have a 1 and a 0 in the target. This step is done focused on the model of not having data that does not have a classification.



In [ ]:
def is_valid_number(x):
    if x == 1 or x == 0:
        return True
    else:
        return True
df[df['target'].apply(lambda x: is_valid_number(x))]

#### Removing punctuations and numbers

In this part we will focus on the text column. Within this column, there is the tweet and it is sought to eliminate the punctuation marks and numbers, which add length to the text but do not add value to the message.


In [ ]:
df['text_clean'] = df['text'].str.replace('[^\w\s]', '', regex=True).str.replace('\d+', '', regex=True)
df.head(5)

#### Removing uppercase

The characters have variations of uppercase and lowercase, but represent the same word. To avoid this situation, in this part of the code for the entire text column we convert the entire text string to lowercase using the "lower" function


In [ ]:
df['text_clean']=df['text_clean'].apply(lambda x: x.lower())
df.head(5)

#### Removing hyperlink

In this part we will focus on the text column. Within this column, there is the tweet and it is sought to eliminate hiperlink, which add length to the text but does not add value to the message. We use the regex library  and replace all the possible hyperlink 



In [ ]:
df['text_clean']=df['text_clean'].apply(lambda x:re.sub('https?://\S+|www\.\S+', '', x))
df.head(5)

#### Removing square brackets

In this part we will focus on the text column. Within this column, there is the tweet and it is sought to eliminate the brackets, which add length to the text but does not add value to the message. We use the regex library  and replace all the possible hyperlink 


In [ ]:
df['text_clean']=df['text_clean'].apply(lambda x:re.sub('\[.*?\]', '', x))
df['text_clean']=df['text_clean'].apply(lambda x:re.sub('<.*?>+', '', x))
df.head(5)

### Feature Engineering

#### Counting the number of words

Using a lambda function, we split the text in the “text clean” column and count how many words there are in each tweet. For example, the tweet with id=1 has 13 words.


In [ ]:
df['word_count'] = df['text_clean'].apply(lambda x: len(x.split()))
df.head(5)

#### Adding the text length 

Similarly, we can calculate how many characters there are in each tweet, applying just the len(x) function. The tweet with id=1 has 68 characters:


In [ ]:
df['size'] = df['text_clean'].apply(lambda x: len(x))
df.head(5)

### Analizing Data

#### Number of Tweets with disasters (1) and non-disasters (0)

We can plot a bar graph to check how many tweets are disasters and how many are not. We take the target column to a new dataframe and plot the graph using seaborn:


In [ ]:
df_target = df['target'].value_counts().to_frame()

## seaborn barplot to display barchart
rcParams['figure.figsize'] = 5, 4
sns.barplot(data=df_target, x=df_target.index, y="target")

We can see that there are more non-disaster tweets (0).

#### Top twenty locations 

Although not every tweet has location information, we can investigate which cities or countries are more represented in the dataset. We calculate the value counts for the column “location” and plot the top 20 results in a bar graph.


In [ ]:
top_location = df["location"].value_counts().head(20)
top_location.plot(kind='bar')

The top locations are “USA”, “New York”, “United States” and “London”. Clearly, USA and United States refer to the same country, and New York is a city in that country. Therefore, it is a good idea to standardize the locations, specially the ones that refer to the same country (USA = United States). However, we will probably leave both cities and countries in our location column, as it is good to have location up to a city scale, even though some of the tweets have only the country and there is no way to find from which city they were written.

#### Histogram of the text size 

In the following histogram, we can see the distribution of the length (characters) of the tweets, which is left-skewed. As expected, there are very few short tweets, and most of them have approximately 120 characters. The maximum number of characters is 140, and therefore we can deduce that this dataset has tweets that were sent before november 2017, when twitter decided to double the character limit from 140 to 280 characters.  

In [ ]:
df['size'].hist()

#### Histogram of the number words

Conversely, the following histogram of the amount of words in the tweets is relatively symmetric, although it has a bit of a longer right tail. We can see that most tweets have between 10 and 18 words.

In [ ]:
df['word_count'].hist()

#### Histogram of the average word length

Finally, we plot a histogram of the average word length in each tweet. We can see that it is a right-skewed distribution and that the average word length in most tweets is 5 to 6 characters. 


In [ ]:
df['text_clean'].str.split().apply(lambda x : [len(i) for i in x]).map(lambda x: np.mean(x)).hist()

### Process data

#### Word tokenization

In [ ]:
AllReviews = df['text_clean']
AllReviews.head()

In [ ]:
review_words_lengths = AllReviews.apply(lambda x: len(nltk.word_tokenize(x)))

#### Sentences with two words

In [ ]:
AllReviews[review_words_lengths[review_words_lengths == 2].index].head(5)

#### Max and min values in the number of words

In [ ]:
print("Minimun number of words", min(review_words_lengths))
print("Maximum number of words", max(review_words_lengths))

#### Histogram of the tokenization words

In [ ]:
## Setting the resolution for better clarity 
review_words_lengths.hist(bins = 25)

The results of the tokenization process show that there is a significant concentration in the groups of tweets that have a number of words between 10 and 20 words. Being the lengths of 13 and 14 where the majority of the tweets are found, with more than 1000 records only in those two lengths. In general terms, it could be said that most of the tweets don’t really correspond to natural disasters, because in the middle of an emergency there is not enough time to write a very long tweet.

#### Histogram of the tokenization words for disaster

In [ ]:
disaster = df[df.target == 1] 
Reviews_disaster = disaster['text_clean'] 
review_words_lengths_disaster = Reviews_disaster.apply(lambda x: len(nltk.word_tokenize(x)))
review_words_lengths_disaster.hist(bins = 25)

This histogram shows what was intuited in the analysis of the previous histogram. Analyzing only the tokenization of tweets that actually correspond to disasters, the distribution of most tweets leans towards shorter lengths than those shown in the histogram without the filtered data.

In conclusion, the precaution that must be taken when training the model must be taken into account, due to the little information that a tweet that corresponds to disaster can contain (remember that the stop words are generating noise when generating the histogram).
In the word cloud diagram that corresponds to positive tweets for disaster, it can be seen that there are keywords that indicate what may be the most recurrent disasters or events in the dataset, such as fires, landslides, terrorist attacks and accidents. Noise can also be evidenced in the data generated by the stop words.


#### Text visualization with word clouds

In [ ]:
def word_cloud_rating(data, target_value, title, columna):
    
    data_filtered = data[data.target == target_value] #filtering according to the star value
    Reviews = data_filtered[columna]

    Reviews_text = ' '.join(Reviews.values) #joining all the words together

    # Creating a word cloud object
    wordcloud = WordCloud(max_font_size=100, max_words=100, background_color="white",scale = 10,width=800, height=400).generate(Reviews_text)

    # Plotting the generated word cloud

    plt.figure(figsize=(8, 19))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.title(title,fontsize=20);
    plt.axis("off")
    plt.show()
    
##### Filtering by target equal 1    
word_cloud_rating(df, 1, 'Disaster Tweets', "text_clean")

##### Filtering by target equal 0    
word_cloud_rating(df, 0, 'Non-Disaster Tweets', "text_clean")

 In the word cloud diagram that corresponds to positive tweets for disaster, it can be seen that there are keywords that indicate what may be the most recurrent disasters or events in the dataset, such as fires, landslides, terrorist attacks and accidents. Noise can also be evidenced in the data generated by the stop words.
As expected, in the tweets that are not related to disasters, there are no words that have a critical meaning or a disaster relationship, for instance, words like people, love, world and time are the most relevant.


#### Most common keywords

In [ ]:
common_keywords = df["keyword"].value_counts().to_frame().head(15)
fig = plt.figure(figsize=(10,4))
sns.barplot(data=common_keywords, x=common_keywords.index, y="keyword",palette="magma")
plt.ylim(38, 46)
plt.title("Most common keywords")
plt.xticks(rotation=55, size=11);

 In the analysis of the distribution of the data in terms of keywords, there is a main word that is transversal to any catastrophic event and that is "fatalities". Then, there are the words related to events that have the component of rain. There are some words that have an important role and that are related to the impact generated by disasters, words like fear, damage, injuries, etc.

#### Most commun words in the tweets

In [ ]:
counter = Counter()
for text in df['text'].values:
    for word in text.split():
        counter[word] += 1
counter.most_common(20)

In [ ]:
data = dict(sorted(counter.items(), key = lambda x: x[1] ,reverse = True)[:10])
words = list(data.keys())
frequency = list(data.values())
frequency

fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (3, 7))
ax.set_facecolor('black')
ax = sns.barplot(x = frequency, y = words, color = '#8699A7', edgecolor = 'white', linewidth = 2)
plt.title('Word Frequency', fontsize = 12)
plt.xlabel('Frequency', fontsize = 12)
plt.ylabel('Words', fontsize = 12)
plt.xticks(size = 12)
plt.yticks(size = 12)
bbox_args = dict(boxstyle = 'round', fc = '0.9')
for p in ax.patches:
    width = p.get_width()
    plt.text(9.5 + p.get_width(), p.get_y() + 0.5 * p.get_height(), '{:1.0f}'.format(width), 
             ha = 'center', 
             va = 'center', 
             color = 'black', 
             bbox = bbox_args, 
             fontsize = 11)
plt.show()

Finally, in the analysis of the frequency of each word, it is evident that the largest amounts are found in the stop words. This confirms what was previously mentioned, when it was said that when these types of words are not removed, noise is created in the data that biases any analysis that one wants to carry out on the composition of the words.

In [ ]:
 #   __  __    ___     ____   _  __  _   _   ____  
 #  |  \/  |  / _ \   / ___| | |/ / | | | | |  _ \ 
 #  | |\/| | | | | | | |     | ' /  | | | | | |_) |
 #  | |  | | | |_| | | |___  | . \  | |_| | |  __/ 
 #  |_|  |_|  \___/   \____| |_|\_\  \___/  |_|    
                                                

<img src="Images/mockup.png"> 

In [ ]:
 #  ___                   ____                   _     _         _____   ____       _    
 # |_ _|  _ __           |  _ \    ___   _ __   | |_  | |__     | ____| |  _ \     / \   
 #  | |  | '_ \   _____  | | | |  / _ \ | '_ \  | __| | '_ \    |  _|   | | | |   / _ \  
 #  | |  | | | | |_____| | |_| | |  __/ | |_) | | |_  | | | |   | |___  | |_| |  / ___ \ 
 # |___| |_| |_|         |____/   \___| | .__/   \__| |_| |_|   |_____| |____/  /_/   \_\
 #                                      |_|                                              

#  Removing stop words in the tweets

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
df['tokenizado'] = df['text_clean'].apply(lambda x: word_tokenize(x))
df.head(3)

In [ ]:
df['text_clean_non_stopwords'] = df['tokenizado'].apply(lambda x: ' '.join([word for word in x if word not in stop_words]))
df.head(3)

###  Comparing disaster vs non-disaster without stop words


In [ ]:
##### Filtering by target equal 1    
word_cloud_rating(df, 1, 'Disaster Tweets', "text_clean_non_stopwords")

##### Filtering by target equal 0    
word_cloud_rating(df, 0, 'Non-Disaster Tweets', "text_clean_non_stopwords")

### Most commun word without stop words

In [ ]:
def palabras_principales(numero_palabras, df, columna, target_value):
    counter = Counter()
    for text in  df[df.target == target_value]["text_clean_non_stopwords"].values:
        for word in text.split():
            counter[word] += 1
    data = dict(sorted(counter.items(), key = lambda x: x[1] ,reverse = True)[:numero_palabras])
    words = list(data.keys())
    frequency = list(data.values())
    return frequency, words

In [ ]:
frequency_dis, words_dis = palabras_principales(20, df, "text_clean_non_stopwords", 1)
frequency_non, words_non = palabras_principales(20, df, "text_clean_non_stopwords", 0)

In [ ]:
fig = plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.barh(words_dis, frequency_dis)
plt.title('Disaster Tweets - Most commun word without stop words')
plt.ylabel('Words')
plt.xlabel('Quantity')

plt.subplot(1, 2, 2)
plt.barh(words_non, frequency_non)
plt.title('NON-Disaster Tweets - Most commun word without stop words')
plt.ylabel('Words')
plt.xlabel('Quantity')
plt.show()

## Feature Engineering for text with non-stopwords

In [ ]:
df['word_count_non_stopwords'] = df['text_clean_non_stopwords'].apply(lambda x: len(x.split()))
df['size_non_stopwords'] = df['text_clean_non_stopwords'].apply(lambda x: len(x))
df.head(3)

In [ ]:
def plot_size(columna1, columna2, title1,title2, maximo, bins_size):
    bins = np.linspace(0, maximo, bins_size)
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 2, 1)
    plt.hist(df[df.target == 1][columna1], bins, alpha=0.5, label='disaster', color ='red')
    plt.hist(df[df.target == 0][columna1], bins, alpha=0.5, label='non_disaster', color ='blue')
    plt.xlabel('Value')
    plt.ylabel('Frecuency')
    plt.legend(loc='upper left')
    plt.title('Histogram of '+title1 )
      
    plt.subplot(1, 2, 2)
    
    plt.hist(df[df.target == 1][columna2], bins, alpha=0.5, label='disaster', color ='red')
    plt.hist(df[df.target == 0][columna2], bins, alpha=0.5, label='non_disaster', color ='blue')
    plt.xlabel('Value')
    plt.ylabel('Frecuency')
    plt.legend(loc='upper left')
    plt.title('Histogram of '+title2 )
    plt.show()

plot_size('word_count_non_stopwords','word_count', 'word count without stopwords','word count with stopwords',25,10)
plot_size('size_non_stopwords','size', 'sensence size without stopwords','sensence with stopwords',140,20)

### BIGRAMS FOR TWEETS (WITH NON STOPWORDS)

In [ ]:
word_tokens = nltk.word_tokenize(''.join(df['text_clean_non_stopwords']))

### Bigram

In [ ]:
bigrams_series = (pd.Series(nltk.ngrams(word_tokens, 2)).value_counts())[:10]
bigrams_series

### Trigram

In [ ]:
trigrams_series = (pd.Series(nltk.ngrams(word_tokens, 3)).value_counts())[:10]
trigrams_series

In [ ]:
f = plt.figure(figsize=(15, 5))
ax1 =  f.add_subplot(121)
bigrams_series.sort_values().plot.barh(color='blue', width=.9)
plt.title('Top-ten Most Frequently Occuring Bigrams')
plt.ylabel('Bigram')
plt.xlabel('# of Occurances')

ax2 = f.add_subplot(122)
trigrams_series.sort_values().plot.barh(color='blue', width=.9)
plt.gca().invert_xaxis()
ax2.yaxis.tick_right()
plt.title('Top-ten Most Frequently Occuring Trigrams')
plt.ylabel('Trigram')
plt.xlabel('# of Occurances')
plt.show()

## Cleaning the country names

### Plot for check the missing values of location's column

In [ ]:
#this plot the color color green represent the missing values all the plot must be have white color. its the reason for try
#extract the location of the text of the tweet.
dfx = pd.read_csv('Data/train.csv', dtype={'id': int, 'keyword': object, 'location': object, 'text': object, 'target': int})
sns.heatmap(dfx.isnull(), cbar=False,cmap="BuGn")

### Special Values

THERE ARE 2 PROBLEMS IN THIS COLUMN (MISSING VALUES AND THE SPACIAL VALUES)

In [ ]:
df['location']=df['location'].copy().astype(str)
print( " "+ str(int(df[df['location']!='nan']['location'].count())) + ' valores diferentes de nan con caracteres especiales')
df[df['location']!='nan']['location'].sort_values(ascending=True)


### Dictonary location and extract the country

In [ ]:
dict_locations = locations.set_index('city').to_dict()['country']
dict_isos=locations.set_index('country').to_dict()['iso3']


### Unique Country

In [ ]:
paisesUnicos = list(np.unique(np.array(list(dict_locations.values()))))

### Extract the country and put the iso (key name)

In [ ]:
#sacar los nombres de una columna y compararlos con la lista de paises.
def devolverPais(frase,textclean):
    valor = None
    
    if 'uk' in frase or 'uk' in textclean :
        valor= 'GBR'
    elif 'usa' in frase or 'usa' in textclean:
        valor= 'USA'
    elif 'us' in frase or 'us' in textclean:
        valor='USA'    
    else:
        for llave in paisesUnicos:
            if str(llave) in str(frase):
                valor = dict_isos[str(llave)] 
                break
   
        if (valor is None):
    
            for llave in dict_locations.keys():
                if str(llave) in str(frase):
                    valor = dict_isos[str(dict_locations[llave])]
                    break
    
    
        if (valor is None):
            for pais in paisesUnicos:
                if str(pais) in str(textclean):
                    valor = dict_isos[str(pais)] 
                    break
   
        if (valor is None):

            for llave in dict_locations.keys():
                if str(llave) in str(textclean):
                    valor = dict_isos[str(dict_locations[llave])]
                    break
            
    return valor 

### Apply the function "devolverPais"

In [ ]:
df["country"] = df['location']
df['country1']=df.apply(lambda x: devolverPais(x.country, x.text_clean_non_stopwords), axis=1)


### Create a new DataFrame (neccesary for graphic)

In [ ]:
paises= df.groupby("country1")["country"].count()
paises=paises.to_frame().reset_index()
paises.rename(columns = {'country':'disasters'}, inplace = True)
paises[paises["country1"]!=None].head()

In [ ]:
import plotly.express as px
fig = px.scatter_geo(paises, locations='country1',
                     color="country1", # which column to use to set the color of markers
                     hover_name="country1", # column added to hover information
                     size="disasters", # size of markers
                     projection="natural earth")
fig.show()

### Image of the map to the PDF view

<img src="Images/map.png"> 

In [ ]:
# ___  ____ ____  _ ____ ____ ___    ____ ____ ___  ____ ____ ___    _  _ ___  ___  ____ ___ ____ 
# |__] |__/ |  |  | |___ |     |     |__/ |___ |__] |  | |__/  |     |  | |__] |  \ |__|  |  |___ 
# |    |  \ |__| _| |___ |___  |     |  \ |___ |    |__| |  \  |     |__| |    |__/ |  |  |  |___ 
                                                                                                                                                                                                                                                               

# Week 9 Project Submission: 

We can see the first two rows of our dataset, which has the cleaned tweet text in the column text_clean

In [ ]:
df.head(2)

We separate each text into words (tokenize) and put all the words into a single list (tokens):

In [ ]:
token_lists = [word_tokenize(each) for each in df.text_clean]
tokens = [item for sublist in token_lists for item in sublist]

In [ ]:
print("Number of unique tokens is: ", len(set(tokens)))

There are 179 english stop words from nltk:

In [ ]:
import nltk
nltk.download('stopwords')

noise_words = []
stopwords_corpus = nltk.corpus.stopwords
eng_stop_words = stopwords_corpus.words('english')
noise_words.extend(eng_stop_words)
print(len(noise_words))
# noise_words


We extract the top 1% words in our dataset (218 words) and we see the first 10:

In [ ]:
one_percentile = int(len(set(tokens)) * 0.01)
top_1_percentile = Counter(tokens).most_common(one_percentile)
one_percentile

In [ ]:
top_1_percentile[:10]

We also see the 10 least common words:

In [ ]:
bottom_1_percentile = Counter(tokens).most_common()[-one_percentile:]
bottom_1_percentile[:10]

We append the top 1% and bottom 1% words to the noise words list:

In [ ]:
noise_words.extend([word for word,val in top_1_percentile])
noise_words.extend([word for word,val in bottom_1_percentile])

In [ ]:
len(noise_words)

Now we have 615 words.

## Building our machine learning model 

For our model, the variables of interest in the dataframe to train the model are the texts and the target as response variables (to be predicted) that are saved with the name "df_model"

In [ ]:
df_model = df[['text_clean', 'target']]
df_model.head(2)

The PorterStemmer and CountVectorizer functions are saved to define a function that can be applied to the text to be analyzed, the stemmer.stem functionality performs a lemmatization of the words, that is, to obtain common roots, the CountVectorizer functionality converts the words (tokens) of the text into a word count sparse matrix.

In [ ]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

The countVectorizer function is given the previously tokenized words, given the noise_words to remove them from parsing, and set n_grams equal to 1 to parse per word.

In [ ]:
# Creating a python object of the class CountVectorizer
bow_counts = CountVectorizer(
    tokenizer=word_tokenize,
    stop_words=noise_words,
    ngram_range=(1, 1)
)

Next, a percentage of the data is specified as a training base and another as a test to start the model training. 80% is defined as a training base and 20% as a test base. In addition, a standardization will be applied to the data set with fit_transform for training and transform for test data.

In [ ]:
reviews_train, reviews_test = train_test_split(df_model, test_size=0.2, random_state=0)

In [ ]:
X_train_bow = bow_counts.fit_transform(reviews_train.text_clean)
X_test_bow = bow_counts.transform(reviews_test.text_clean)

In [ ]:
y_train_bow = reviews_train['target']
y_test_bow = reviews_test['target']

In [ ]:
y_test_bow.value_counts() / y_test_bow.shape[0]

### Applying logistic regression

Once the text data has been cleaned, they are entered into a first test model, where logistic regression will be used as a data training model and the model is adjusted.

In [ ]:
lr_model_all = LogisticRegression(C=0.9, solver="liblinear") #c = float
lr_model_all.fit(X_train_bow, y_train_bow)

To evaluate the model's classification performance, the precision metrics F1-score and Accuracy will be reviewed, which are established from the true positives and negatives and false positives and negatives of the confusion matrix as follows:

<img src=https://miro.medium.com/max/1400/1*Tmbqxgs51wdK4mwbXJbtoA.png>

To evaluate the performance of the model, the following measures are calculated $$F1score:2\frac{precision.recal}{precision+recall}$$ Where $$presicion:\frac{TP}{TP+FP} \quad recall:\frac{TP}{TP+FN}$$ And $$ Accuracy:\frac{TP+TN}{TP+TN+FP+FN}$$

The **F1 score** value is used to combine the precision and recall measures into a single value and the **Accuracy** measures the percentage of cases that the model has got right.

In [ ]:
# Predicting the output
test_pred_lr_prob = lr_model_all.predict_proba(X_test_bow)
test_pred_lr_all = lr_model_all.predict(X_test_bow)

print("F1 score: ", f1_score(y_test_bow, test_pred_lr_all))
print("Accuracy: ", accuracy_score(y_test_bow, test_pred_lr_all) * 100)

With our initial model we obtained these percentages of performance, adjustments and other models will be evaluated to achieve performance metrics closer to 1

Next, the probabilities obtained by applying the logistic model are obtained and a dataframe is built by joining the original data with the Target (Response Variable) and the prediction column thrown by the model and the associated probability are added in order to perform a review of misclassifications and see the range of probabilities associated with classification 0 and 1 in the model.

In [ ]:
probabilities = [each[1] for each in test_pred_lr_prob]

In [ ]:
predictions = pd.DataFrame()
predictions['Text'] = reviews_test['text_clean']
predictions['Target'] = reviews_test['target']
predictions['Predicted_dissaster'] = test_pred_lr_all
predictions['Predicted_probability'] = probabilities

In [ ]:
predictions.head(10)

the next steps are:
    1.train the model (logistic regresion)
    2.remove de noise words and 
    3.identify the words that describe a disaster and what words don't describe this event (through the weight)

In [ ]:
lr_model_all_new = LogisticRegression(C=0.9, solver="liblinear")
lr_model_all_new

In [ ]:
# Training the model
lr_model_all_new.fit(X_train_bow, y_train_bow)

# Predicting the results
test_pred_lr_prob = lr_model_all_new.predict_proba(X_test_bow)
test_pred_lr_all = lr_model_all_new.predict(X_test_bow)

In [ ]:
lr_weights = pd.DataFrame(list(
    zip(
        bow_counts.get_feature_names(),
        lr_model_all_new.coef_[0])
    ),
    columns=['words','weights']
)

lr_weights.sort_values(['weights'],ascending = False)[:15]

In [ ]:
lr_weights.sort_values(['weights'],ascending = False)[-15:]

# Other parameters, noise words removed

In [ ]:
# Changes with respect to the previous code
# 1. Increasing the n-grams from just having 1-gram to (1-gram, 2-gram,3-gram and 4-gram)
# 2. Including the stopwords in the bag of words features

bow_counts = CountVectorizer(
    tokenizer=word_tokenize,
    ngram_range=(1,4)
)

X_train_bow = bow_counts.fit_transform(reviews_train.text_clean)
X_test_bow = bow_counts.transform(reviews_test.text_clean)

In [ ]:
lr_model_all_new = LogisticRegression(C=0.9, solver="liblinear")
lr_model_all_new

In [ ]:
# Training the model
lr_model_all_new.fit(X_train_bow, y_train_bow)

# Predicting the results
test_pred_lr_prob = lr_model_all_new.predict_proba(X_test_bow)
test_pred_lr_all = lr_model_all_new.predict(X_test_bow)

print("F1 score: ", f1_score(y_test_bow, test_pred_lr_all))
print("Accuracy: ", accuracy_score(y_test_bow, test_pred_lr_all) * 100)

The accuracy has jumped from to 80.8%. This is an example of what simple hyperparameter tuning and input feature modification can do to the overall performance. We can even get interpretable features from this in terms of what contributed the most to positive and negative sentiment:

### Palabras asociadas a comentarios de desastres o no desastres

We can even get interpretable features in terms of what contributed the most to positive and negative sentiment:

in this firt dataframe we can see that the 5 most important words that made match with the distar are: "hiroshima, fire,california, earthquake and floods"

In [ ]:
lr_weights = pd.DataFrame(list(
    zip(
        bow_counts.get_feature_names(),
        lr_model_all_new.coef_[0])
    ),
    columns=['words','weights']
)

lr_weights.sort_values(['weights'],ascending = False)[:5]

in the second dataframe we can see that the 5 most important words that did not match with the distar are: "im, new, screams, body, you"

In [ ]:
lr_weights.sort_values(['weights'],ascending = False)[-5:]

### Fandom forests classification

In [ ]:
rf_model_all = RandomForestClassifier(n_estimators=200)

# Training the data
rf_model_all.fit(X_train_bow, y_train_bow)

# Generating predictions
test_pred_lr_prob = rf_model_all.predict_proba(X_test_bow)
test_pred_lr_all = rf_model_all.predict(X_test_bow)


In [ ]:
print("F1 score: ", f1_score(y_test_bow,test_pred_lr_all))
print("Accuracy: ", accuracy_score(y_test_bow,test_pred_lr_all)* 100)

the accuracy with the random forest is lower, this show us that there aren't perfects models, the best way to find the best model is try y compare differents models.

## TF-IDF model 

We employ the **Term Frequency-Inverse Document Frequency (TF-IDF)** method to evaluates how important a word is into a tweet. 

The relevance of a word grows according to the number of times it appears in the tweet, but this is mitigated by the frequency of the term in the corpus.

For our case, the TF-IDF weight is the product of two terms. The first computes the normalized Term Frequency (TF); i.e. the number of times a word appears in a tweet divided by the total number of words in that all tweets. The second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the tweets  divided by the number of tweets where the specific term appears:

TF-IDF doesn't just count each word - it applies a weighting so that common words receive less attention and rare words receive more.

Were-featurize our original set of tweets based on TF-IDF and split the resulting features into train and test sets a:

In [ ]:
# Create a vectorizer - we still feed in our stop words, although
# these are less relevant now as TF-IDF would weight them less anyway.
tfidf_counts = TfidfVectorizer(
    tokenizer=word_tokenize,
    stop_words=noise_words,
    ngram_range=(1,3)
)

X_train_tfidf = tfidf_counts.fit_transform(reviews_train.text_clean)
X_test_tfidf = tfidf_counts.transform(reviews_test.text_clean)

#### Applying logistic regression to TF-IDF features

We applied logistic regression to the features created from TF-IDF:

In [ ]:
# Create the classifier
lr_model_tf_idf = LogisticRegression(solver="liblinear")

# Train the classifier
lr_model_tf_idf.fit(X_train_tfidf, y_train_bow)

# Predict the results
test_pred_lr_prob = lr_model_tf_idf.predict_proba(X_test_tfidf)
test_pred_lr_all = lr_model_tf_idf.predict(X_test_tfidf)

## Evaluating the model
print("F1 score: ",f1_score(y_test_bow, test_pred_lr_all))
print("Accuracy: ", accuracy_score(y_test_bow, test_pred_lr_all) * 100)

We achieved an accuracy of ***77.3*** percent with ***TF-IDF*** vs ***78.26*** percent with ***1-grams***. In this case a more advanced vectorizing method produces inferior results, although it's possible that penalizing terms that are prevalent across the tweets disadvantages this specific sample. 

### Increasing the model accuracy

We modified the parameters of the logistic regression as

* Setting ngram_range=***(1,4)*** in the Vectorizer
* Not removing the noise words beforehand in the Vectorizer
* Setting ***C=10*** in the LogisticRegression classsifier
* Setting ***penalty=l1*** in the LogisticRegression classifier

In [ ]:
# Changes: Removing stop words and including 1-4 grams in the tf-idf data
tfidf_counts = TfidfVectorizer(
    tokenizer=word_tokenize,
    ngram_range=(1,4)
)

#Applying the transformations
X_train_tfidf = tfidf_counts.fit_transform(reviews_train.text_clean)
X_test_tfidf = tfidf_counts.transform(reviews_test.text_clean)

In [ ]:
# Setting up the model class
lr_model_tf_idf_new = LogisticRegression(solver="liblinear", penalty='l1', C=10)

# Training the model 
lr_model_tf_idf_new.fit(X_train_tfidf, y_train_bow)

# Prediciting the results
test_pred_lr_prob = lr_model_tf_idf_new.predict_proba(X_test_tfidf)
test_pred_lr_all = lr_model_tf_idf_new.predict(X_test_tfidf)

In [ ]:
## Evaluating the model
print("F1 score: ",f1_score(y_test_bow, test_pred_lr_all))
print("Accuracy: ", accuracy_score(y_test_bow, test_pred_lr_all) * 100)

This is not an improvement over our prior result (just 0.1). Moreover,  we made four modifications four modifications at the same time, so we don't know which ones benefited.

We employ new hyperparameters to enhance your model, which is known as **hyperparameter tuning**. 

We can also find our most important features again, as below:

In [ ]:
lr_weights = pd.DataFrame(
    list(
        zip(tfidf_counts.get_feature_names(), lr_model_tf_idf_new.coef_[0])
    ),
    columns=['words','weights']
)

lr_weights.sort_values(['weights'],ascending = False)[:10]

In [ ]:
lr_weights.sort_values(['weights'],ascending = False)[-10:]

## Word embeddings model 

Finally, we are going to use the word embedding model, that is a type type of word representation that allows words with similar meaning to have a similar representation. 

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML

There are different methods to learn word embeddings. For our case we will use a model that is part of word2vec, one of the most famous methods.

In [ ]:
import gensim
import os

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format(
    os.path.join(os.getcwd(), 'extra_Data/glove.twitter.27B.200d_out.txt'),
    binary=False,
    unicode_errors='ignore'
)

In [ ]:
print("The embedding for great is", len(model['great']), "dimensional")

Now, we are going to calculate the vector for every single tweet in our dataset

In [ ]:
review_embeddings = []

for each_review in df.text_clean:
    review_average = np.zeros(model.vector_size)
    count_val = 0
    
    for each_word in word_tokenize(each_review):

        # Change to "if True" to remove stop words from the 
        # averaged embeddings
        if False:
            if(each_word.lower() in noise_words):
                print(each_word.lower())
                continue
        
        if(each_word.lower() in model):
            review_average += model[each_word.lower()]
            count_val += 1
    
    review_embeddings.append(list(review_average/count_val))

Then, we converts the list of vector representations for each tweet into a new dataframe ans split into train and test sets


In [ ]:
embedding_data = pd.DataFrame(review_embeddings)
embedding_data = embedding_data.fillna(0)

In [ ]:
embedding_data

In [ ]:
X_train_embed, X_test_embed, y_train_embed, y_test_embed =  train_test_split(
    embedding_data,
    df.target,
    test_size=0.2,
    random_state=0
)

Now, we applied logistic regression for our word embedding representation 

In [ ]:
lr_model = LogisticRegression(penalty="l1", C=10, solver="liblinear")
lr_model.fit(X_train_embed, y_train_embed)
test_pred_lr_prob = lr_model.predict_proba(X_test_embed)
test_pred_lr_all = lr_model.predict(X_test_embed)

print("F1 score: ", f1_score(y_test_embed, test_pred_lr_all))
print("Accuracy: ", accuracy_score(y_test_embed, test_pred_lr_all)*100)

As we can see, we increase the accuracy of our model. It's not enough but is a great advance in order to build a 100% accuracy model to predict the specific situation

## NEW MODEL

## Summary
- First of all, we imported all the libraries that would be useful to analyze the information of the datasets (pandas, numpy, matplotlib, etc)

- Then, we uploaded the two datasets (train and test datasets) into our notebook and we gave a basic description of it, describing its columns and summarizing some important information for our case, like the most repetitive target in the data (0: not disaster)

- Thirdly, we cleaned the data, in order to extract the most valuable information from it. We removed rows, which do not have 1 and 0 values in target column, punctuations, numbers, uppercase, hyperlinks and square brackets.

- For the feature engineering section, we added some useful columns such as: number of words in the tweets and text length (total characters). We found out that this dataset was probably made before 2017, when the tweet limit was 140 characters. Regarding location, we saw that it is important to standarize some words that are different but refer to the same country (USA and United States for example).

- Finally, we analyze the data, using some useful visualizations such as: histograms, barplots and word clouds graphs, focusing on the most repetitive words. Here we found an important opportunity with “stop words” that are popular in most of the tweets, but without a context do not give us a lot of information.


### Repository
https://github.com/oscarciceri/Disaster_Monitoring_Based_On_Tweets/blob/main/EDA_extended.ipynb